In [1]:
%load_ext autoreload
%autoreload all
import numpy as np
import pandas as pd
import pandas_ta as ta
from faker import Faker
from custom_providers import dates, names_and_tickers, price_columns, sectors
from calculated_columns.moving_average import calculate_moving_average
from calculated_columns.macd import calculate_macd
import talib

In [2]:
# Specify number of rows and price range
num_rows = 100
price_range = (100, 1000)
time_period_in_days = 8

# Add custom providers
fake = Faker()
fake.add_provider(dates.Dates)
fake.add_provider(names_and_tickers.NamesAndTickers)
fake.add_provider(price_columns.PriceColumns)
fake.add_provider(sectors.Sectors)

In [3]:
# Generate values for initial columns
date_df = fake.generate_dates(time_period_in_days=time_period_in_days,
                              num_rows=num_rows,)

# name_prob_dict = {"Company": 0.1, "Ticker": 0.1}
name_ticker_df = fake.generate_names_and_tickers(
    time_period_in_days=time_period_in_days,
    num_rows=num_rows
)#, **name_prob_dict)

sector_df = fake.generate_sectors(
    num_rows=num_rows,
    time_period_in_days=time_period_in_days
)

# price_prob_dict = {
#     "Open Price": 0.1,
#     "Close Price": 0.1,
#     "High Price": 0.1,
#     "Low Price": 0.1,
#     "Volume": 0.1
# }

price_df = fake.generate_prices(
    num_rows=num_rows,
    price_range=price_range
)

In [4]:
df = pd.concat([date_df, name_ticker_df, sector_df, price_df], axis=1)
df.head()

,Date,Company,Ticker,Sector,Open Price,Close Price,High Price,Low Price,Volume
0,2023-01-19,Jones Inc,JI2E8,Consumer Goods,141.397455,139.889615,144.257289,124.659403,241915
1,2023-01-20,Jones Inc,JI2E8,Consumer Goods,131.514632,128.909800,134.374466,113.679588,737952
2,2023-01-21,Jones Inc,JI2E8,Consumer Goods,112.707259,116.794285,119.654119,97.477046,218124
3,2023-01-22,Jones Inc,JI2E8,Consumer Goods,144.616730,148.530827,151.390660,129.386517,447477
4,2023-01-23,Jones Inc,JI2E8,Consumer Goods,277.884390,285.471934,288.331767,262.654177,535830


In [5]:
window = 10

# Calculate and add Moving Average with numpy
df['Moving Average'] = calculate_moving_average(df["Close Price"], window)

# Relative Strength Index with pandas_ta
df['RSI'] = ta.rsi(df['Close Price'], length=14)

# Calculate and add MACD and Signal Line columns to df
calculate_macd(df, "Close Price")

# Bollinger Bands with TALib
df['SMA'] = talib.SMA(df['Close Price'], timeperiod=window)
df['Upper_Band'], df['Middle_Band'], df['Lower_Band'] = talib.BBANDS(df['Close Price'], timeperiod=window, nbdevup=2, nbdevdn=2)

# On-Balance Volume (OBV) with TALib
df['OBV'] = talib.OBV(df['Close Price'], df['Volume'])

#Chaikin Money Flow with TALib
df['CMF'] = talib.ADOSC(df['High Price'], df['Low Price'], df['Close Price'], df['Volume'], fastperiod=3, slowperiod=10)

In [7]:
# Calculate RSI
rsi_period = 14  # RSI period, typically 14 days
rsi_values = talib.RSI(df['Close Price'], timeperiod=rsi_period)

# Add RSI values to the DataFrame
df['rsi'] = rsi_values

In [7]:
len(df.columns)

21